In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train = pd.read_csv("train.csv")
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [7]:
Y = train.target.copy()

In [9]:
X = train[[el for el in train.keys() if el != "target"]]

In [31]:
X

,id,keyword,location,text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...
...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...


In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=.075,random_state=42)

In [15]:
X_train.text.apply(lambda t: len(t.split())).quantile([.8,.85,.90,.95])

0.80    20.0
0.85    21.0
0.90    22.0
0.95    24.0
Name: text, dtype: float64

In [16]:
tokenizer = Tokenizer(num_words=1500,oov_token="<OOV>")

In [17]:
tokenizer.fit_on_texts(X.text)

In [18]:
train_sequences = tokenizer.texts_to_sequences(X_train.text)
test_sequences = tokenizer.texts_to_sequences(X_test.text)

In [26]:
train_sequences_padded = pad_sequences(train_sequences,maxlen=22,truncating='post',padding='post')
test_sequences_padded = pad_sequences(test_sequences,maxlen=22,truncating='post',padding='post')

In [29]:
#lets standardize it
MEAN = train_sequences_padded.mean()
STD = train_sequences_padded.std()

In [30]:
train_sequences_standardized = (train_sequences_padded-MEAN)/STD
test_sequences_standardized = (test_sequences_padded-MEAN)/STD

In [33]:
print(train_sequences_standardized.shape,y_train.shape)

(7042, 22) (7042,)


In [56]:
#build basic model
dense_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(22,1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048,activation='relu'),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [57]:
dense_model.compile(optimizer="Adam",loss=tf.keras.losses.binary_crossentropy,metrics=['acc'])

In [58]:
dense_model.fit(train_sequences_standardized,y_train,batch_size = 512,epochs=100,validation_data=(test_sequences_standardized,y_test))

Epoch 1/100
14/14 [==============================] - 2s 128ms/step - loss: 0.6781 - acc: 0.5825 - val_loss: 0.6567 - val_acc: 0.6165
Epoch 2/100
14/14 [==============================] - 2s 115ms/step - loss: 0.6513 - acc: 0.6252 - val_loss: 0.6522 - val_acc: 0.6200
Epoch 3/100
14/14 [==============================] - 2s 119ms/step - loss: 0.6395 - acc: 0.6403 - val_loss: 0.6560 - val_acc: 0.6235
Epoch 4/100
14/14 [==============================] - 2s 119ms/step - loss: 0.6263 - acc: 0.6487 - val_loss: 0.6538 - val_acc: 0.6165
Epoch 5/100
14/14 [==============================] - 2s 148ms/step - loss: 0.6173 - acc: 0.6583 - val_loss: 0.6670 - val_acc: 0.5972
Epoch 6/100
14/14 [==============================] - 2s 128ms/step - loss: 0.6016 - acc: 0.6680 - val_loss: 0.6575 - val_acc: 0.6112
Epoch 7/100
14/14 [==============================] - 2s 117ms/step - loss: 0.5808 - acc: 0.6903 - val_loss: 0.6647 - val_acc: 0.6112
Epoch 8/100
14/14 [==============================] - 2s 116ms/step - 

In [69]:
num_of_dims = 10
embedded_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1500,num_of_dims,input_length=22),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [70]:
embedded_model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc'])

In [71]:
embedded_model.fit(train_sequences_padded,y_train,epochs=10,validation_data=(test_sequences_padded,y_test))

Epoch 1/10
221/221 [==============================] - 1s 2ms/step - loss: 0.6429 - acc: 0.6314 - val_loss: 0.5743 - val_acc: 0.7198
Epoch 2/10
221/221 [==============================] - 0s 2ms/step - loss: 0.4846 - acc: 0.7802 - val_loss: 0.4887 - val_acc: 0.7898
Epoch 3/10
221/221 [==============================] - 0s 1ms/step - loss: 0.4049 - acc: 0.8266 - val_loss: 0.4829 - val_acc: 0.7863
Epoch 4/10
221/221 [==============================] - 0s 1ms/step - loss: 0.3746 - acc: 0.8421 - val_loss: 0.4873 - val_acc: 0.8021
Epoch 5/10
221/221 [==============================] - 0s 1ms/step - loss: 0.3558 - acc: 0.8490 - val_loss: 0.4936 - val_acc: 0.7986
Epoch 6/10
221/221 [==============================] - 0s 1ms/step - loss: 0.3425 - acc: 0.8559 - val_loss: 0.5022 - val_acc: 0.7863
Epoch 7/10
221/221 [==============================] - 0s 1ms/step - loss: 0.3338 - acc: 0.8584 - val_loss: 0.5138 - val_acc: 0.7776
Epoch 8/10
221/221 [==============================] - 0s 1ms/step - loss: 0.